In [ ]:
import cv2
from ultralytics import YOLO
import numpy as np
import serial

In [ ]:
# init the serial for arduino
arduino = serial.Serial(port='/dev/cu.usbserial-0001',
                        baudrate=9600, timeout=1)

# Function to send data


def send_data(data):
    if arduino.isOpen():
        arduino.write(data.encode())  # Send data as bytes
        print(f"Sent: {data}")
    else:
        print("Serial port is not open!")

In [ ]:
import time  # Import time module for tracking intervals

# Load the trained YOLOv8 model
model = YOLO('best.pt')

# Initialize webcam
cap = cv2.VideoCapture(0)

# Get screen center
ret, frame = cap.read()
height, width, _ = frame.shape
center_screen = (width // 2, height // 2)

# Define class names and assign colors
class_names = ['customer', 'talking', 'eye contact']
colors = {
    'customer': (0, 255, 255),  # Cyan for regular faces
    'talking': (0, 255, 0),     # Green
    'eye contact': (0, 0, 255)  # Red
}
target_color = (255, 0, 0)  # Red for the target face

# Time tracking for 3-second interval
last_sent_time = time.time()

while True:
    ret, frame = cap.read()
    # frame = cv2.flip(frame, 1)
    if not ret:
        break

    # Perform detection
    results = model.predict(source=frame, conf=0.5)

    closest_face = None
    min_distance = float('inf')
    closest_face_center = None  # To store center coordinates of the closest face

    # Draw detections on the frame
    for box in results[0].boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # Get bounding box coordinates
        conf = box.conf[0]  # Confidence score
        label = int(box.cls)  # Class label as integer
        # Retrieve class name using the label as index
        class_name = class_names[label]

        # Calculate the center of the bounding box
        center_x = (x1 + x2) // 2
        center_y = (y1 + y2) // 2

        # Draw the bounding box and label on the frame
        color = colors[class_name]
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, f'{class_name} {conf:.2f}', (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Check if the detected class is "customer"
        if class_name == 'customer':
            # Calculate distance to the center of the screen
            distance = np.sqrt(
                (center_x - center_screen[0]) ** 2 + (center_y - center_screen[1]) ** 2)
            if distance < min_distance:
                min_distance = distance
                closest_face = (x1, y1, x2, y2, conf, class_name)
                # Update the closest center
                closest_face_center = (center_x, center_y)

    # Highlight the closest face and show center coordinates
    if closest_face:
        x1, y1, x2, y2, conf, class_name = closest_face
        center_x, center_y = closest_face_center  # Retrieve the center coordinates
        cv2.rectangle(frame, (x1, y1), (x2, y2), target_color, 2)
        cv2.putText(frame, f'Target {conf:.2f}', (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, target_color, 2)
        # Display the coordinates of the center of the target face
        cv2.putText(frame, f'Center: ({center_x}, {center_y})', (center_x + 10, center_y + 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        # Print the center coordinates to the console
        print(f'Target Center Coordinates: ({center_x}, {center_y})')

        # Send the center coordinates every 3 seconds
        current_time = time.time()
        if current_time - last_sent_time >= 2:  # Check if 3 seconds have passed
            try:
                send_data(f"{center_x},{center_y}")
                print(f"Sent Data: {center_x},{center_y}")
                last_sent_time = current_time  # Update the last sent time
            except Exception as e:
                print(f"Error: {e}")

    # Display the frame with detections
    cv2.imshow('Real-Time Detection', frame)

    # Break on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
! pip3 install --extra-index-url https://google-coral.github.io/py-repo/ tflite_runtime

In [ ]:
! pip3 install mediapipe

In [ ]:
"""
Tflite installer:
pip3 install --extra-index-url https://google-coral.github.io/py-repo/ tflite_runtime

"""


import cv2
import mediapipe as mp  # type: ignore
import numpy as np
# import socket
import time  # Added for interval implementation
import serial

# init the serial for arduino
arduino = serial.Serial(port='/dev/cu.usbserial-0001',
                        baudrate=9600, timeout=1)

# Function to send data


def send_data(data):
    if arduino.isOpen():
        arduino.write(data.encode())  # Send data as bytes
        print(f"Sent: {data}")
    else:
        print("Serial port is not open!")


# Mediapipe initializations
mp_face_detection = mp.solutions.face_detection
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils


def calculate_dominance_score(face_area, eye_stability, mouth_movement):
    """
    Calculate a dominance score based on normalized face area, eye stability, and mouth movement.
    """
    normalized_face_area = face_area / 40000  # Adjust based on resolution
    dominance_score = normalized_face_area * 0.1 + \
        eye_stability * 0.1 + mouth_movement * 0.8
    return dominance_score


def detect_and_track_faces():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Unable to access the camera.")
        return

    # Socket setup
    server_address = ('127.0.0.1', 65432)
    # sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    try:
        # sock.connect(server_address)
        print("Socket connection established.")
    except ConnectionRefusedError:
        print("Error: Unable to connect to the server.")
        return

    with mp_face_detection.FaceDetection(min_detection_confidence=0.5) as face_detection, \
            mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=10, min_detection_confidence=0.5) as face_mesh:

        try:
            last_sent_time = 0  # Initialize timestamp for the interval
            while True:
                ret, frame = cap.read()
                if not ret:
                    print("Error: Unable to read the frame.")
                    break

                # frame = cv2.flip(frame, 1)
                height, width, _ = frame.shape
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                face_results = face_detection.process(rgb_frame)
                dominant_face = None
                dominant_score = -1
                dominant_coords = (0, 0)

                if face_results.detections:
                    for detection in face_results.detections:
                        bboxC = detection.location_data.relative_bounding_box
                        x, y, w, h = (
                            int(bboxC.xmin * width),
                            int(bboxC.ymin * height),
                            int(bboxC.width * width),
                            int(bboxC.height * height),
                        )
                        face_area = w * h
                        face_center = (x + w // 2, y + h // 2)

                        # Eye and Mouth Tracking
                        face_mesh_results = face_mesh.process(rgb_frame)
                        eye_stability, mouth_movement = 0, 0

                        if face_mesh_results.multi_face_landmarks:
                            for landmarks in face_mesh_results.multi_face_landmarks:
                                # Eye Tracking
                                left_eye_indices = [
                                    33, 160, 158, 133, 153, 144]
                                right_eye_indices = [
                                    362, 385, 387, 263, 373, 380]

                                left_eye_coords = [(landmarks.landmark[i].x * width, landmarks.landmark[i].y * height)
                                                   for i in left_eye_indices]
                                right_eye_coords = [(landmarks.landmark[i].x * width, landmarks.landmark[i].y * height)
                                                    for i in right_eye_indices]

                                left_eye_stability = np.std(
                                    [pt[1] for pt in left_eye_coords])
                                right_eye_stability = np.std(
                                    [pt[1] for pt in right_eye_coords])
                                eye_stability = max(
                                    0, 10 - (left_eye_stability + right_eye_stability) / 2)

                                # Mouth Tracking
                                mouth_indices = [78, 308, 13, 14, 87, 317]
                                mouth_coords = [(landmarks.landmark[i].x * width, landmarks.landmark[i].y * height)
                                                for i in mouth_indices]

                                mouth_diff = np.mean(
                                    [abs(pt[1] - mouth_coords[0][1]) for pt in mouth_coords])
                                mouth_movement = min(15, mouth_diff)

                        # Calculate dominance score
                        dominance_score = calculate_dominance_score(
                            face_area, eye_stability, mouth_movement)

                        # Update dominant face
                        if dominance_score > dominant_score + 0.6:
                            dominant_score = dominance_score
                            dominant_face = face_center
                            dominant_coords = (x, y, w, h)

                    # Send dominant face coordinates via socket at 1-second intervals
                    current_time = time.time()
                    if dominant_face and current_time - last_sent_time >= 2:

                        message = f"{dominant_face[0]},{dominant_face[1]}"
                        # sock.sendall(message.encode('utf-8'))
                        last_sent_time = current_time  # Update last sent time
                        send_data(message)

                        # Highlight dominant face
                        x, y, w, h = dominant_coords
                        cv2.rectangle(
                            frame, (x, y), (x + w, y + h), (0, 255, 255), 2)
                        cv2.putText(frame, f"Dominant Face - Coords: {dominant_face}, Score: {dominant_score:.2f}",
                                    (x, y - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

                        # Print coordinates and score to the terminal
                        print(
                            f"Dominant Face Coords: {dominant_face}, Dominance Score: {dominant_score:.2f}")

                # Display the frame
                cv2.imshow("Dominant Face Tracking", frame)

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

        finally:
            # sock.close()
            cap.release()
            cv2.destroyAllWindows()


if __name__ == "__main__":
    detect_and_track_faces()